# Projet - Classification de documents d'opinions
---
- Un jeu de données textuelles est mis à disposition sur Moodle.  
- Il s'agit d'un corpus d'à peu près 8000 documents contenant des avis d'internautes sur des films.  
- A chaque document est associé sa polarité selon l'avis (+1 : positif, -1 : négatif).  
- Le fichier des documents est formaté dans un tableau cvs (un avis par ligne), un autre fichier csv contient les polarités d'avis par document (- 1/+1).  
- Une correspondance directe existe entre les numéros des lignes des documents et des polarités.

## Configuration de l'environnement
---

In [1]:
import pandas as pd
import numpy as np

from collections import defaultdict

from nltk import pos_tag
from nltk import punkt
from nltk.corpus import stopwords, wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn import model_selection, naive_bayes, svm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

#import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('word_tokenize')
#nltk.download('averaged_perceptron_tagger')

# Set seed for random results base calculation
np.random.seed(500)

import seaborn as sns
sns.set(style = "darkgrid")

## Lecture des fichiers
---

- Récupération du fichier dataset.csv et labels.csv
- Intégration de leur contenu dans 2 dataframes pandas différents

In [2]:
print("\nDataframe des avis")
df_avis = pd.read_csv('Dataset/dataset.csv', sep = '\n', header = None, names = ['Avis'])
display(df_avis.head())
display(df_avis.shape)

print("\nDataframe des scores")
df_score = pd.read_csv('Dataset/labels.csv', sep = '\n', header = None, names = ['Score'])
display(df_score.head())
display(df_score.shape)

print("\nDataframe merged")
df = df_avis.join(df_score)
display(df.head())
display(df.shape)


Dataframe des avis


,Avis
0,Obviously made to show famous 1950s stripper M...
1,This film was more effective in persuading me ...
2,Unless you are already familiar with the pop s...
3,From around the time Europe began fighting Wor...
4,Im not surprised that even cowgirls get the bl...


(10000, 1)


Dataframe des scores


,Score
0,-1
1,-1
2,-1
3,-1
4,-1


(10000, 1)


Dataframe merged


,Avis,Score
0,Obviously made to show famous 1950s stripper M...,-1
1,This film was more effective in persuading me ...,-1
2,Unless you are already familiar with the pop s...,-1
3,From around the time Europe began fighting Wor...,-1
4,Im not surprised that even cowgirls get the bl...,-1


(10000, 2)

## Pré-traitement des données
---

Structuration des données afin d'y réaliser des manipulations

### Tokenization

In [3]:
# Copie du DF originale
df2 = shuffle(df)
# Réinitialisation des index
#df2.reset_index()
df2.reset_index(inplace = True, 
                drop = True)

display(df2.head(20))

,Avis,Score
0,After having read two or three negative review...,1
1,I recently (May 2008) discovered that this chi...,1
2,"Pathetic is the word. Bad acting, pathetic scr...",-1
3,Spencer Tracy and Katherine Hepburn would roll...,-1
4,This in my opinion is one of the best action m...,1
5,I can't believe this show is still rating a 9 ...,-1
6,"This is a truly awful ""B"" movie. It is witless...",-1
7,I bought Bloodsuckers on ebay a while ago. I w...,-1
8,I saw this film first on my way home from Pari...,1
9,This 1986 Italian-French remake of the 1946 fi...,1


In [13]:

# Change all characters to lower case
df2['Avis'] = [entry.lower() for entry in df['Avis']]

# Tokenization
df2['Avis'] = [word_tokenize(entry) for entry in df['Avis']]

display(df2.head())

,Avis,Score
0,"[Obviously, made, to, show, famous, 1950s, str...",1
1,"[This, film, was, more, effective, in, persuad...",1
2,"[Unless, you, are, already, familiar, with, th...",-1
3,"[From, around, the, time, Europe, began, fight...",-1
4,"[Im, not, surprised, that, even, cowgirls, get...",1


In [15]:
# Lemmenting

## liste d'étiquettes
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

# version rapide
df2_small = df2.iloc[:100]

for index, entry in enumerate(df2_small['Avis']):
    final_words = []
    word_lem = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_final = word_lem.lemmatize(word, tag_map[tag[0]])
            final_words.append(word_final)
    df2_small.loc[index, 'Avis'] = str(final_words)

display(df2_small.head())

,Avis,Score
0,"['Obviously', 'make', 'show', 'famous', 'strip...",1
1,"['This', 'film', 'effective', 'persuade', 'Zio...",1
2,"['Unless', 'already', 'familiar', 'pop', 'star...",-1
3,"['From', 'around', 'time', 'Europe', 'begin', ...",-1
4,"['Im', 'surprise', 'even', 'cowgirls', 'get', ...",1


# To-Do

- prepare train/test data sets
- encode
- word vect
- try algorithms (Naive Bayes and SVM)